In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.linalg import sqrtm

In [2]:
# MSE error
def calc_error(r, target):
    return r - target

def calc_loss(r, target):
    return 0.5 * ((r - target)**2).mean()

def calc_accuracy_onehot(r, target):
    # one hot encoding of output
    b = np.zeros_like(r)
    b[np.where(r == np.amax(r))] = 1
    return np.linalg.norm(b * target)

def calc_accuracy_single(r, target):
    if r >= 0.5:
        return target * 1.0
    else:
        return 1.0 - target

def linear(x):
    return x

def d_linear(x):
    return np.ones_like(x)

def relu(x):
    return np.maximum(x, 0, x)

def d_relu(x):
    return np.heaviside(x, 0)

def logistic(x):
    return 1/(1 + np.exp(-x))

def d_logistic(x):
    y = logistic(x)
    return y * (1.0 - y)

def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    y = tanh(x)
    return 1 - y**2

def ds_pinv(r_array, w_matrix):

    covariance = np.cov(r_array.T)
    mean = np.mean(r_array, axis=0)
    gammasquared = covariance + np.outer(mean,mean)
    
    gamma = sqrtm(gammasquared)

    gen_pseudo = np.dot(gamma, np.linalg.pinv(np.dot(w_matrix, gamma)))
    if not np.all(np.isreal(gen_pseudo) == True):
        raise ValueError('ds-pinv has not converged to real matrix')

    return gen_pseudo

def cos_sim(A, B):
    return np.trace(A.T @ B) / np.linalg.norm(A) / np.linalg.norm(B)

def sem(A):
    return np.std(A) / np.sqrt(len(A))

In [3]:
def calc_net(lr, layers, activation, d_activation, W_init, inputs, steps, algo, learn_W, bias, learn_bias, acc_measure, bw_lr = None, alpha = None, pinv_recalc = 4, print_steps = 10):

    number_layers = len(layers)
    e = [[] for layer in range(number_layers)]
    e_array = []
    loss_array = []
    acc_array = []
    r = [[] for layer in range(number_layers)]
    W_array = []
    B_array = []
    bias_array = []
    r_array = []
    bias_array.append(copy.deepcopy(bias))

    W = copy.deepcopy(W_init)
    
    W_array.append(copy.deepcopy(W_init))
    
    if algo == 'fa':
        B = []
        for mat in W_init:
            B.append(np.random.randn(*mat.shape))
    
    elif algo in ['pbp', 'gen-pbp', 'dyn-pbp']:
        pinv_counter = 0
        B = []
        for mat in W_init:
            B.append(np.linalg.pinv(mat))

    for i in range(steps):
        if print_steps > 0:
            if i % (steps / print_steps) == 0:
                print("Working on step " , i)
        dW = [np.zeros_like(W) for W in W_init]
        dB = [np.zeros_like(W.T) for W in W_init]
        dbias = [np.zeros_like(bias) for bias in bias]
        
        # batch learn over all samples
        for j in range(len(inputs)):
            r[0] = inputs[j]
            target = targets[j]

            # fw pass
            for l in range(len(layers)-2):
                r[l+1] = activation(W[l] @ r[l] + bias[l])
            r[-1] = W[-1] @ r[-2] + bias[-1]

            # bw pass
            e[-1] = calc_error(r[-1], target)

            for k in range(len(layers)-2, -1, -1):
                if algo == 'bp':
                    e[k] = np.diag(d_activation(r[k])) @ W[k].T @ e[k+1]
                elif algo == 'fa':
                    e[k] = np.diag(d_activation(r[k])) @ B[k].T @ e[k+1]
                elif algo in ['pbp', 'gen-pbp', 'dyn-pbp']:
                    e[k] = np.diag(d_activation(r[k])) @ B[k] @ e[k+1]

            # calculate weight update
            for l in range(len(layers)-1):
                if learn_W[l]:
                    dW[l] += np.outer(e[l+1], r[l])
                if learn_bias[l]:
                    dbias[l] += e[l+1]
                if algo == 'dyn-pbp':
                    dB[l] += np.outer(B[l] @ W[l] @ r[l] - r[l], W[l] @ r[l])
                    
            r_array.append(r.copy())           
            e_array.append(e.copy())
            W_array.append(W.copy())
            if algo in ['pbp', 'gen-pbp', 'dyn-pbp']:
                B_array.append(B.copy())
            bias_array.append(copy.deepcopy(bias))
            loss_array.append(calc_loss(r[-1], target))
            
            if acc_measure == "onehot":
                acc_array.append(calc_accuracy_onehot(r[-1], target))
            elif acc_measure == "single":
                acc_array.append(calc_accuracy_single(r[-1][0], target))
                    
        # update weights
        for l in range(len(layers)-1):
            if learn_W[l]:
                W[l] -= lr * dW[l]
            if learn_bias[l]:
                bias[l] -= lr * dbias[l]
            if algo == 'dyn-pbp':
                B[l] -= bw_lr * (dB[l] - alpha * B[l])
        
        if algo in ['pbp', 'gen-pbp']:
            pinv_counter += 1
            if pinv_counter == pinv_recalc:
                for k in range(len(layers)-2, -1, -1):
                    if algo == 'pbp':
                        B[k] = np.linalg.pinv(W[k])
                    elif algo == 'gen-pbp':
                        # create a batch of r's since last recalc of backwards weights
                        r_batch = [r_array[-pinv_recalc:][i][k] for i in range(pinv_recalc)]
                        # catch an exeption where ds-pinv does not converge to a real matrix
                        try:
                            B[k] = ds_pinv(np.array(r_batch), W[k])
                        except ValueError:
                            print('ds-pinv has not converged to real matrix')
                            return _, _, _, _, _, _
                    pinv_counter = 0
    if algo in ['pbp', 'gen-pbp', 'dyn-pbp']:
        return r_array, e_array, W_array, B_array, loss_array, acc_array
    else:
        return r_array, e_array, W_array, _, loss_array, acc_array

In [4]:
# plot options
labels = ['bp', 'fa', 'pbp', 'gen-pbp', 'dyn-pbp']
styles = ['g', 'b', 'r', 'y', 'black']

# The setup

 We use single-target encoding.
 
 Also, bias is deactivated because we want to stress how FA is not able to produce sign changes in the error signal.
 
 Therefore, we need a net of hidden layer size 3, i.e. 2-3-1. (?)
 
 A note on pinv_recalc:
 Here, we sample over all 4 examples given by xor. Therefore, pinv-recalc should be set to 4 or multiples of 4.

# Comparison of activation functions

Let's start by comparing the effect of different activation functions with backprop.

In [5]:
# define parameters of model
lr = 0.1
layers = [2, 3, 1]

inputs = np.array([[1,0], [0,1], [1,1], [0,0]])
targets = np.array([1, 1, 0, 0])

steps = 10000
seeds = 10

In [6]:
for activation in [[relu, d_relu], [logistic, d_logistic], [tanh, d_tanh]]:
    acc = []
    for seed in range(seeds):

        np.random.seed(seed)
        W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
        W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
        bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
        W_init = [W1_init, W2_init]

        _, _, _, _, _, acc_array_bp = calc_net(lr, layers, activation[0], activation[1], W_init, inputs, steps, 'bp', learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0)

        acc.append(np.mean(acc_array_bp[-100:]))
        print('accuracy over last 100 samples: ', acc[-1])

    print('--------------------------------------')
    print('activation: ', str(activation[0]),', total accuracy over ', seeds, ' seeds: ', np.mean(acc), ' +- ', np.std(acc)/np.sqrt(seeds))

accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  0.25
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  0.5
--------------------------------------
activation:  <function relu at 0x7fbb49189790> , total accuracy over  10  seeds:  0.75  +-  0.07071067811865475
accuracy over last 100 samples:  0.25
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.5
accuracy over last 100 samples:  0.75
accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.5
accuracy over last 100 samples:  0.0
accuracy over last 100 samples:  1.0
accuracy over last 100 samples:  0.75
--------------------------------------
activation:  <function logistic at 0x7fbb491898b0> , total a

## Conclusion:

Tanh gives best results for this initialization and learning rate, so we pick this.

# Comparison of models

First, let's run a parameter scan for each model with 10 seeds

In [17]:
# define parameters of model
lrs = [0.001, 0.01, 0.1]

layers = [2, 3, 1]

inputs = np.array([[1,0], [0,1], [1,1], [0,0]])
targets = np.array([1, 1, 0, 0])

steps = 2000
seeds = 20

In [16]:
for model in ['fa']:
    for lr in [0.1]:
        acc = []
        for seed in range(20):

            np.random.seed(seed)
            W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
            W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
            bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
            W_init = [W1_init, W2_init]

            _, _, _, _, _, acc_array = calc_net(0.1, layers, tanh, d_tanh, W_init, inputs, steps, 'fa',
                                            learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1),
                                            acc_measure = 'single', print_steps=0, pinv_recalc=4)

            acc.append(acc_array)
        print('model:', model, 'lr:', lr, 'accuracy:', np.mean(np.array(acc_array[-100:])), '+-', np.std(np.array(acc_array[-100:]))/np.sqrt(seeds))

model: fa lr: 0.1 accuracy: 0.25 +- 0.13693063937629152


In [14]:
acc = []
for seed in range(20):

    np.random.seed(seed)
    W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
    W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
    bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
    W_init = [W1_init, W2_init]

    _, _, _, _, _, acc_array = calc_net(0.1, layers, tanh, d_tanh, W_init, inputs, steps, 'fa',
                                    learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1),
                                    acc_measure = 'single', print_steps=0, pinv_recalc=4)

    acc.append(acc_array)
print('model:', model, 'lr:', lr, 'accuracy:', np.mean(np.array(acc_array[-100:])), '+-', np.std(np.array(acc_array[-100:]))/np.sqrt(seeds))

model: fa lr: 0.1 accuracy: 0.25 +- 0.13693063937629152


In [18]:
for model in ['bp', 'fa', 'pbp', 'gen-pbp']:
    for lr in lrs:
        acc = []
        for seed in range(seeds):

            np.random.seed(seed)
            W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
            W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
            bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
            W_init = [W1_init, W2_init]

            _, _, _, _, _, acc_array = calc_net(lr, layers, tanh, d_tanh, W_init, inputs, steps, model,
                                            learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1),
                                            acc_measure = 'single', print_steps=0, pinv_recalc=4)

            acc.append(acc_array)
        print('model:', model, 'lr:', lr, 'accuracy:', np.mean(np.array(acc_array[-100:])), '+-', np.std(np.array(acc_array[-100:]))/np.sqrt(seeds))

model: bp lr: 0.001 accuracy: 0.25 +- 0.09682458365518541
model: bp lr: 0.01 accuracy: 0.75 +- 0.09682458365518541
model: bp lr: 0.1 accuracy: 1.0 +- 0.0
model: fa lr: 0.001 accuracy: 0.5 +- 0.11180339887498948
model: fa lr: 0.01 accuracy: 0.75 +- 0.09682458365518541
model: fa lr: 0.1 accuracy: 0.25 +- 0.09682458365518541
model: pbp lr: 0.001 accuracy: 0.25 +- 0.09682458365518541
model: pbp lr: 0.01 accuracy: 0.75 +- 0.09682458365518541
model: pbp lr: 0.1 accuracy: 1.0 +- 0.0
model: gen-pbp lr: 0.001 accuracy: 0.25 +- 0.09682458365518541
model: gen-pbp lr: 0.01 accuracy: 0.75 +- 0.09682458365518541
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
ds-pinv has not converged to real matrix
model: gen-pbp lr: 0.1 accuracy: 0.1251119550779376 +- 0.019778757

In [21]:
# for model dyn-pseudo, we also sample over backwards learning rate

bw_lrs = [1e-5, 1e-4, 1e-3]

for lr in lrs:
    for bw_lr in bw_lrs:
        acc = []
        for seed in range(seeds):

            np.random.seed(seed)
            W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
            W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
            bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
            W_init = [W1_init, W2_init]

            _, _, _, _, _, acc_array = calc_net(lr, layers, tanh, d_tanh, W_init, inputs, steps, 'dyn-pbp',
                                            learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1),
                                            acc_measure = 'single', print_steps=0, pinv_recalc=1, alpha = 0.1, bw_lr = bw_lr)

            acc.append(acc_array)
        print('model:', 'dyn-pbp', 'lr:', lr, 'bw_lr:', bw_lr, 'accuracy:', np.mean(np.array(acc_array[-100:])), '+-', np.std(np.array(acc_array[-100:]))/np.sqrt(seeds))

model: dyn-pbp lr: 0.001 bw_lr: 1e-05 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.001 bw_lr: 0.0001 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.001 bw_lr: 0.001 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.01 bw_lr: 1e-05 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.01 bw_lr: 0.0001 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.01 bw_lr: 0.001 accuracy: 0.5 +- 0.07071067811865475
model: dyn-pbp lr: 0.1 bw_lr: 1e-05 accuracy: 1.0 +- 0.0
model: dyn-pbp lr: 0.1 bw_lr: 0.0001 accuracy: 1.0 +- 0.0
model: dyn-pbp lr: 0.1 bw_lr: 0.001 accuracy: 1.0 +- 0.0


From this, we use the best values and rerun for more seeds:

In [27]:
# define parameters of model
lr_bp = 0.1
lr_fa = 0.01
lr_pbp = 0.1
lr_gen = 0.01
lr_dyn = 0.1
bw_lr = 1e-3

layers = [2, 3, 1]

inputs = np.array([[1,0], [0,1], [1,1], [0,0]])
targets = np.array([1, 1, 0, 0])

steps = 2000
seeds = 50

In [ ]:
acc = []
loss = []

# for dyn-pseudo, we also record the backwards arrays
B_array = []
W_array = []
r_array = []

for seed in range(seeds):
    
    np.random.seed(seed)
    W1_init = np.random.normal(0, 1, size=(layers[1], layers[0]))
    W2_init = np.random.normal(0, 1, size=(layers[2], layers[1]))
    bias_init = [np.zeros(layers[1]), np.zeros(layers[2])]
    W_init = [W1_init, W2_init]
    
    _, _, _, _, loss_array_bp, acc_array_bp = calc_net(lr_bp, layers, tanh, d_tanh, W_init, inputs, steps, 'bp', learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0)
    _, _, _, _, loss_array_fa, acc_array_fa = calc_net(lr_fa, layers, tanh, d_tanh, W_init, inputs, steps, 'fa', learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0)
    _, _, _, _, loss_array_pbp, acc_array_pbp = calc_net(lr_pbp, layers, tanh, d_tanh, W_init, inputs, steps, 'pbp', learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0, pinv_recalc=4)
    _, _, _, _, loss_array_gen, acc_array_gen = calc_net(lr_gen, layers, tanh, d_tanh, W_init, inputs, steps, 'gen-pbp', learn_W = [True]*(len(layers)-1), bias = bias_init, learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0, pinv_recalc=4)
    r, _, W, B, loss_array_dyn, acc_array_dyn = calc_net(lr_dyn, layers, tanh, d_tanh, W_init, inputs, steps, 'dyn-pbp', learn_W = [True]*(len(layers)-1), bias = bias_init,
                                                      learn_bias = [False]*(len(layers)-1), acc_measure = 'single', print_steps=0, pinv_recalc=1, alpha = 0.1, bw_lr = bw_lr)
    
    B_array.append(B)
    W_array.append(W)
    r_array.append(r)
    acc.append([acc_array_bp, acc_array_fa, acc_array_pbp, acc_array_gen, acc_array_dyn])
    loss.append([loss_array_bp, loss_array_fa, loss_array_pbp, loss_array_gen, loss_array_dyn])

In [ ]:
print('total accuracy over', seeds, 'seeds: ')

labels = ['bp', 'fa', 'pbp', 'gen-pbp', 'dyn-pbp']


for i in range(len(labels)):
    print(labels[i], ':', np.mean(np.transpose(np.array(acc), axes=(1,0,2))[i]), ' +- ',  sem(np.mean(np.transpose(np.array(acc), axes=(1,0,2))[0], axis = 1)))


In [ ]:
per_steps = 1

for i in range(len(labels)):
    data = np.transpose(np.array(loss), axes = (1,0,2))[i]
    plt.plot(np.linspace(0, len(data[0][::per_steps]), len(data[0][::per_steps])), np.mean(data, axis=0)[::per_steps], label=labels[i], c=styles[i])
    
plt.yscale('log')
plt.xlabel('Step')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(len(labels), 1, figsize=(10,10))

per_steps = 1

for i in range(len(labels)):
    data = np.transpose(np.array(acc), axes = (1,0,2))[i]
    ax[i].plot(np.linspace(0, len(data[0][::per_steps]), len(data[0][::per_steps])), np.mean(data, axis=0)[::per_steps], label=labels[i], c=styles[i])
    ax[i].legend(loc='lower right')

plt.xlabel('Step')
fig.text(0.06, 0.5, 'Accuracy', ha='center', va='center', rotation='vertical')
plt.show()

In [17]:
cos_array = []
for layer in range(len(B_array[0][0])):
    temp = []
    for seed in range(len(B_array)):
        temp.append([cos_sim(ds_pinv(r_array[seed][step][layer], W[seed][step][layer]), B_array[seed][step][layer]) for step in range(steps)])
        

ValueError: shapes (2,) and (1,1) not aligned: 2 (dim 0) != 1 (dim 0)